### Import Labraries

In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GATConv
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from tqdm import tqdm
from scipy.spatial.distance import cosine

In [4]:
#Load Features
# ----------------------------------------

features_df = pd.read_csv("../SavedFeatures/Feature_P_16_RGB_ADL_Sampled.csv")
train_df, test_df = train_test_split(features_df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)
def process_data(df):
    X = torch.tensor(df.iloc[:, 3:].values, dtype=torch.float32)
    y = torch.tensor(df["ActionLabel"].values, dtype=torch.long)
    return X, y

X_train, y_train = process_data(train_df)
X_val, y_val = process_data(val_df)
X_test, y_test = process_data(test_df)

In [5]:
features_df

,Frame,ActionLabel,ActionName,Feature_0,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,...,Feature_2038,Feature_2039,Feature_2040,Feature_2041,Feature_2042,Feature_2043,Feature_2044,Feature_2045,Feature_2046,Feature_2047
0,frame_00000.jpg,0,Unknown,0.104695,0.256306,0.334437,0.182553,0.507135,0.074522,0.409498,...,0.060401,0.062081,0.058476,0.145272,0.218072,0.008137,0.289075,0.072365,0.427008,0.229325
1,frame_00005.jpg,0,Unknown,0.087917,0.254906,0.297352,0.225716,0.472780,0.037875,0.330567,...,0.083597,0.043358,0.043557,0.108556,0.120676,0.010515,0.231653,0.084674,0.372557,0.194336
2,frame_00010.jpg,0,Unknown,0.078217,0.186071,0.256348,0.182095,0.462404,0.056442,0.313948,...,0.051538,0.045468,0.044084,0.125724,0.119578,0.017788,0.207659,0.109599,0.475324,0.203805
3,frame_00015.jpg,0,Unknown,0.118012,0.138689,0.278767,0.231952,0.355594,0.058108,0.491719,...,0.039603,0.036425,0.075125,0.141178,0.176244,0.026528,0.288970,0.156908,0.417525,0.240268
4,frame_00020.jpg,0,Unknown,0.161606,0.089609,0.185414,0.306446,0.394624,0.152874,0.469531,...,0.017577,0.026200,0.113280,0.090700,0.152429,0.018096,0.336645,0.180360,0.551914,0.276418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5035,frame_25175.jpg,0,Unknown,0.356032,0.407906,0.318134,0.264215,2.619350,0.226759,0.033055,...,0.569711,0.275820,0.221272,0.179127,0.695197,0.062038,0.195177,0.089863,0.516486,0.008244
5036,frame_25180.jpg,0,Unknown,0.288469,0.351928,0.256074,0.201457,2.101087,0.150330,0.026395,...,0.802069,0.264970,0.301837,0.182716,0.437268,0.033345,0.066309,0.111978,0.503713,0.009151
5037,frame_25185.jpg,0,Unknown,0.461195,0.213695,0.287767,0.257083,1.840744,0.080183,0.091401,...,0.647266,0.269381,0.271354,0.124471,0.696608,0.108354,0.162290,0.181776,0.447647,0.044464
5038,frame_25190.jpg,0,Unknown,0.425387,0.474241,0.216757,0.186205,2.028681,0.167828,0.143815,...,0.738819,0.318604,0.392267,0.136912,0.632361,0.122288,0.107961,0.209704,0.315855,0.055593


### DPT

In [6]:
def dynamic_graph_construction(X, k=3, device='cpu'):
    X = X.to(device)
    N = X.shape[0]
    sim_matrix = F.cosine_similarity(X.unsqueeze(1), X.unsqueeze(0), dim=2)

    _, top_k_indices = torch.topk(sim_matrix, k=k+1, dim=1)

    row_indices = torch.arange(N).repeat_interleave(k).to(device)
    col_indices = top_k_indices[:, 1:].reshape(-1)
    edge_index = torch.stack([row_indices, col_indices], dim=0)

    return edge_index

### GAT - Loss Function

In [7]:
class GATModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, heads=2):
        super(GATModel, self).__init__()
        self.conv1 = GATConv(input_dim, hidden_dim, heads=heads)
        self.conv2 = GATConv(hidden_dim * heads, output_dim, heads=1)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

class MultiTaskLoss(nn.Module):
    def __init__(self, alpha=0.8):
        super(MultiTaskLoss, self).__init__()
        self.ce_loss = nn.CrossEntropyLoss()
        self.alpha = alpha

    def forward(self, outputs, targets, features):
        ce = self.ce_loss(outputs, targets)
        temporal_loss = torch.mean(torch.abs(features[1:] - features[:-1]))
        return self.alpha * ce + (1 - self.alpha) * temporal_loss


### Model

In [9]:
input_dim = X_train.shape[1]
hidden_dim = 128
output_dim = features_df['ActionLabel'].nunique()
model = GATModel(input_dim, hidden_dim, output_dim)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)
criterion = MultiTaskLoss()

### Training

In [10]:
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    edge_index = dynamic_graph_construction(X_train, k=3).to(device)
    optimizer.zero_grad()
    output = model(X_train.to(device), edge_index)
    loss = criterion(output, y_train.to(device), X_train.to(device))
    loss.backward()
    optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        edge_index_val = dynamic_graph_construction(X_val, k=3).to(device)
        val_output = model(X_val.to(device), edge_index_val)
        val_loss = criterion(val_output, y_val.to(device), X_val.to(device))
        val_probs = torch.softmax(val_output, dim=1)
        val_preds = val_probs.argmax(dim=1).cpu().numpy()
        # Top-1 and Top-5 Accuracy
        top1_correct = (val_preds == y_val.cpu().numpy()).sum()
        top1_acc = top1_correct / len(y_val)  # Top-1 Accuracy
        top5_preds = torch.topk(val_probs, 5, dim=1).indices.cpu().numpy()
        top5_correct = sum([y in top5 for y, top5 in zip(y_val.cpu().numpy(), top5_preds)])
        top5_acc = top5_correct / len(y_val)  # Top-5 Accuracy
        val_accuracy = accuracy_score(y_val.cpu().numpy(), val_preds)

    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {loss.item():.4f} - Val Loss: {val_loss.item():.4f} - Val Acc: {val_accuracy:.4f} - Top-1 Acc: {top1_acc:.4f} - Top-5 Acc: {top5_acc:.4f}")



RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 85201920000 bytes.

### Confusion Matrix

In [ ]:
if epoch == num_epochs - 1:  # Only display for the last epoch
    from sklearn.metrics import confusion_matrix
    import matplotlib.pyplot as plt
    import seaborn as sns

    conf_matrix = confusion_matrix(y_val.cpu().numpy(), val_preds)
    plt.figure(figsize=(15, 7))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Reds')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix - Validation Set')
    plt.show()